In [54]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd

### Split dataset to train test

In [34]:
df_original = pd.read_csv('../datasets/loan_with_features.csv')
assert df_original.shape[0] > 1

In [35]:
# shuffle dataset
df = df_original.sample(frac=1, random_state=42)
assert not df.iloc[0].equals(df_original.iloc[0])

In [36]:
m = df.shape[0]
assert m > 1

train_df = df.iloc[:round(m * 0.8)]
test_df = df.iloc[round(m * 0.8):]
assert train_df.shape[0] + test_df.shape[0] == m

In [37]:
train_df.to_csv('../datasets/train.csv', index=False)
test_df.to_csv('../datasets/test.csv', index=False)

### Training

In [38]:
train_data = TabularDataset('../datasets/train.csv')
train_data.head()

Loaded data from: ../datasets/train.csv | Columns = 63 / 63 | Rows = 231 -> 231


,loan_amount,loan_duration,loan_payments,loan_status,account_frequency,disp_type,card_type,gender,age,district_name,...,avg_monthly_amount_p12m,avg_monthly_balance_p12m,trans_type_prijem_count,trans_type_vyber_count,trans_type_vydaj_count,trans_operation_prevod_na_ucet_count,trans_operation_prevod_z_uctu_count,trans_operation_vklad_count,trans_operation_vyber_count,trans_operation_vyber_kartou
0,65184,12,5432.0,A,POPLATEK MESICNE,OWNER,gold,M,37,Brno - mesto,...,61360.250000,243060.750000,30.0,4.0,48.0,NaN,15.0,1.0,52.0,NaN
1,29784,24,1241.0,A,POPLATEK MESICNE,OWNER,NaN,F,26,Plzen - mesto,...,33328.000000,132604.000000,18.0,NaN,6.0,NaN,NaN,12.0,6.0,NaN
2,94512,12,7876.0,B,POPLATEK MESICNE,OWNER,NaN,M,45,Ostrava - mesto,...,101782.909091,292740.272727,63.0,7.0,64.0,NaN,21.0,2.0,71.0,NaN
3,7656,24,319.0,A,POPLATEK MESICNE,OWNER,NaN,F,32,Rychnov nad Kneznou,...,80210.461538,578773.384615,72.0,NaN,90.0,16.0,NaN,28.0,74.0,NaN
4,113796,36,3161.0,A,POPLATEK PO OBRATU,DISPONENT,NaN,F,18,Brno - mesto,...,77439.166667,442085.500000,50.0,8.0,86.0,32.0,24.0,4.0,62.0,NaN


In [39]:
hyperparameters = {
	'NN_TORCH': [{}],
	# 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
	'CAT': [{}],
	# 'XGB': [{}],
	'FASTAI': [{}],
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
}
predictor = (TabularPredictor(label='loan_status').fit(train_data, hyperparameters=hyperparameters))

No path specified. Models will be saved in: "AutogluonModels/ag-20250420_103422"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.0
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          8
Memory Avail:       4.50 GB / 16.00 GB (28.1%)
Disk Space Avail:   59.56 GB / 233.47 GB (25.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Reco

### Prediction

In [40]:
test_data = TabularDataset('../datasets/test.csv')

y_pred = predictor.predict(test_data.drop(columns=['loan_status']))
y_pred.head()

Loaded data from: ../datasets/test.csv | Columns = 63 / 63 | Rows = 58 -> 58


0    A
1    A
2    A
3    A
4    A
Name: loan_status, dtype: object

In [41]:
predictor.evaluate(test_data, silent=True)

/Users/vincentcheng/Documents/data_science/loan-prediction-app-aws/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.8793103448275862,
 'balanced_accuracy': 0.5000000000000001,
 'mcc': 0.0,
 'roc_auc': 0.8137254901960784,
 'f1': 0.0,
 'precision': 0.0,
 'recall': 0.0}

In [42]:
predictor.leaderboard(test_data)

/Users/vincentcheng/Documents/data_science/loan-prediction-app-aws/.venv/lib/python3.10/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsUnif,0.896552,0.872340,accuracy,0.002380,0.154433,0.012347,0.002380,0.154433,0.012347,1,True,1
1,KNeighborsDist,0.879310,0.851064,accuracy,0.001689,0.003000,0.015813,0.001689,0.003000,0.015813,1,True,2
2,CatBoost,0.879310,0.936170,accuracy,0.008362,0.007109,1.523464,0.008362,0.007109,1.523464,1,True,5
3,NeuralNetTorch,0.879310,0.893617,accuracy,0.027971,0.025487,0.473429,0.027971,0.025487,0.473429,1,True,9
4,RandomForestEntr,0.879310,0.936170,accuracy,0.048003,0.038582,0.641859,0.048003,0.038582,0.641859,1,True,4
5,RandomForestGini,0.879310,0.936170,accuracy,0.052552,0.042986,0.549225,0.052552,0.042986,0.549225,1,True,3
6,ExtraTreesEntr,0.879310,0.914894,accuracy,0.053428,0.041098,0.482298,0.053428,0.041098,0.482298,1,True,7
7,WeightedEnsemble_L2,0.879310,0.936170,accuracy,0.053913,0.043710,0.616541,0.001361,0.000724,0.067316,2,True,10
8,ExtraTreesGini,0.879310,0.936170,accuracy,0.054972,0.057892,0.462971,0.054972,0.057892,0.462971,1,True,6
9,NeuralNetFastAI,0.844828,0.936170,accuracy,0.021219,0.014919,1.368694,0.021219,0.014919,1.368694,1,True,8


### Feature Selection

In [43]:
from autogluon.tabular import TabularPredictor
import pandas as pd

In [44]:
predictor = TabularPredictor.load('AutogluonModels/ag-20250419_082259')
test_data = pd.read_csv('../datasets/test.csv')

In [45]:
feature_importance = predictor.feature_importance(
  data=test_data
)

Computing feature importance via permutation shuffling for 62 features using 58 rows with 5 shuffle sets...


/Users/vincentcheng/Documents/data_science/loan-prediction-app-aws/.venv/lib/python3.10/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
	13.09s	= Expected runtime (2.62s per shuffle set)
/Users/vincentcheng/Documents/data_science/loan-prediction-app-aws/.venv/lib/python3.10/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, 

In [46]:
feature_importance

,importance,stddev,p_value,n,p99_high,p99_low
trans_date_latest,0.158621,0.061443,0.002236,5,0.285133,0.032108
loan_duration,0.055172,0.014425,0.000513,5,0.084874,0.025471
card_type,0.037931,0.014425,0.002091,5,0.067633,0.008229
trans_amount_latest,0.037931,0.014425,0.002091,5,0.067633,0.008229
trans_operation_vklad_count,0.037931,0.022480,0.009777,5,0.084218,-0.008356
...,...,...,...,...,...,...
avg_amount_p3m,-0.003448,0.007711,0.813050,5,0.012428,-0.019324
avg_monthly_balance_p3m,-0.003448,0.007711,0.813050,5,0.012428,-0.019324
avg_weekly_balance_p6m,-0.003448,0.007711,0.813050,5,0.012428,-0.019324
avg_balance_p12m,-0.006897,0.015421,0.813050,5,0.024856,-0.038649


In [47]:
feature_importance.query('p_value <= 0.05')

,importance,stddev,p_value,n,p99_high,p99_low
trans_date_latest,0.158621,0.061443,0.002236,5,0.285133,0.032108
loan_duration,0.055172,0.014425,0.000513,5,0.084874,0.025471
card_type,0.037931,0.014425,0.002091,5,0.067633,0.008229
trans_amount_latest,0.037931,0.014425,0.002091,5,0.067633,0.008229
trans_operation_vklad_count,0.037931,0.022480,0.009777,5,0.084218,-0.008356
loan_payments,0.034483,0.024383,0.017055,5,0.084688,-0.015722
trans_count_p3m,0.034483,0.027261,0.023710,5,0.090614,-0.021648
loan_amount,0.024138,0.009443,0.002318,5,0.043582,0.004694
median_daily_trans_count_p3m,0.024138,0.019658,0.025803,5,0.064614,-0.016339
avg_monthly_amount_p3m,0.020690,0.007711,0.001941,5,0.036566,0.004813


In [55]:
features_selected = [
  'trans_date_latest',
  'loan_duration',
  'district_name',
  'loan_payments',
  'loan_amount',
  'trans_balance_latest',
  'median_daily_trans_count_p3m',
  'trans_amount_latest',
]

### Train new model with selected features

In [49]:
train_data = pd.read_csv('../datasets/train.csv')
test_data = pd.read_csv('../datasets/test.csv')

In [50]:
train_data = train_data[features_selected + ['loan_status']]
test_data = test_data[features_selected + ['loan_status']]

In [51]:
hyperparameters = {
	'NN_TORCH': [{}],
	# 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
	'CAT': [{}],
	# 'XGB': [{}],
	'FASTAI': [{}],
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
}
predictor = (TabularPredictor(
  label='loan_status',
  path='AutogluonModels/final'
  )
  .fit(
    train_data=train_data, 
    hyperparameters=hyperparameters,
		presets='optimize_for_deployment'
  )
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.0
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          8
Memory Avail:       4.48 GB / 16.00 GB (28.0%)
Disk Space Avail:   59.55 GB / 233.47 GB (25.5%)
Presets specified: ['optimize_for_deployment']
Beginning AutoGluon training ...
AutoGluon will save models to "/Users/vincentcheng/Documents/data_science/loan-prediction-app-aws/jupyter-nb/AutogluonModels/final"
Train Data Rows:    231
Train Data Columns: 8
Label Column:       loan_status
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['A', 'B']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_typ

In [52]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.896551724137931,
 'balanced_accuracy': 0.5714285714285714,
 'mcc': 0.35751859933740576,
 'roc_auc': 0.8235294117647058,
 'f1': 0.25,
 'precision': 1.0,
 'recall': 0.14285714285714285}

In [53]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.896552,0.93617,accuracy,0.003869,0.001886,0.779409,0.003869,0.001886,0.779409,1,True,1
1,WeightedEnsemble_L2,0.896552,0.93617,accuracy,0.006572,0.002844,0.854800,0.002703,0.000958,0.075391,2,True,2


### Test New Model

In [63]:
train_data = pd.read_csv('../datasets/train.csv')
test_data = pd.read_csv('../datasets/test.csv')
predictor = TabularPredictor.load('AutogluonModels/final')

In [58]:
test_data[features_selected].head()

,trans_date_latest,loan_duration,district_name,loan_payments,loan_amount,trans_balance_latest,median_daily_trans_count_p3m,trans_amount_latest
0,1994-09-30,12,Brno - venkov,5955.0,71460,76863,2.0,142
1,1994-05-05,12,Praha - zapad,6695.0,80340,89343,2.0,40840
2,1995-07-07,24,Praha - vychod,4108.0,98592,62672,1.0,2700
3,1994-07-31,12,Brno - venkov,1219.0,14628,34903,1.0,15
4,1996-07-21,24,Olomouc,7796.0,187104,86878,1.0,46500


In [70]:
test_data[features_selected + ['loan_status']].query('loan_status == "B"')

,trans_date_latest,loan_duration,district_name,loan_payments,loan_amount,trans_balance_latest,median_daily_trans_count_p3m,trans_amount_latest,loan_status
6,1997-08-05,12,Trebic,4280.0,51360,64530,1.0,29972,B
9,1996-04-11,12,Tabor,2523.0,30276,24930,1.0,18500,B
17,1993-09-21,24,Karlovy Vary,7281.0,174744,58141,1.0,40175,B
40,1993-12-17,24,Brno - mesto,3151.0,75624,41955,1.0,13500,B
45,1995-01-01,36,Hl.m. Praha,7518.0,270648,50997,1.0,25800,B
50,1994-11-12,24,Strakonice,8023.0,192552,14203,1.0,8023,B
54,1994-07-07,36,Breclav,7685.0,276660,19783,1.0,4987,B


In [67]:
predictor.predict(test_data[features_selected])

0     A
1     A
2     A
3     A
4     A
5     A
6     A
7     A
8     A
9     A
10    A
11    A
12    A
13    A
14    A
15    A
16    A
17    A
18    A
19    A
20    A
21    A
22    A
23    A
24    A
25    A
26    A
27    A
28    A
29    A
30    A
31    A
32    A
33    A
34    A
35    A
36    A
37    A
38    A
39    A
40    A
41    A
42    A
43    A
44    A
45    A
46    A
47    A
48    A
49    A
50    B
51    A
52    A
53    A
54    A
55    A
56    A
57    A
Name: loan_status, dtype: object